In [19]:
import pandas as pd
import re
from lxml import etree
import shutil
import os
import math
import copy

In [20]:
def get_mapping_for_xml_invoice(file_path_of_sample_xml):

    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:Invoice-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2"
    #     'xmlns' : "urn:oasis:names:specification:ubl:schema:xsd:Invoice-2"
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
    # format_key_map = []
    invoice_dict={}
    for element in target_root.iter():
        ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

        string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

        path = ' / '.join(string_ancestors)
        for prefix, uri in namespace_map.items():
            path = path.replace('{' + uri + '}', prefix + ':')
        if 'sb:StandardBusinessDocument / ubl:Invoice / ' in path:
            path = path.replace('sb:StandardBusinessDocument / ', '')
            if element.attrib:
                xpath_base = path
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e

                    invoice_dict[xpath] = tree.getpath(element)
            else:
                invoice_dict[path] = tree.getpath(element)

        else:
            if element.attrib:
                xpath_base = get_element_xpath(element)
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
    #                 print(f'-->{xpath}')
                    invoice_dict[xpath] = tree.getpath(element)
            else:
                xpath = get_element_xpath(element)#defined on top
                invoice_dict[xpath] = tree.getpath(element)
    r_h = invoice_dict
    return r_h
def get_mapping_for_xml_credit(file_path_of_sample_xml):
    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:CreditNote-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2" 
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
        # format_key_map = []
    credit_dict={}
    for element in target_root.iter():
        if isinstance(element, etree._Comment):
            continue

        else:
            ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

            string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

            path = ' / '.join(string_ancestors)
            for prefix, uri in namespace_map.items():
                path = path.replace('{' + uri + '}', prefix + ':')

            if 'sb:StandardBusinessDocument / ubl:CreditNote / ' in path:

                path = path.replace('sb:StandardBusinessDocument / ', '')

                if element.attrib:
                    xpath_base = path
                    credit_dict[xpath_base] = tree.getpath(element)
                    for e,val in element.attrib.items():
                        xpath = xpath_base + ' / ' + '@' + e

                        credit_dict[xpath] = tree.getpath(element)
                else:
                    credit_dict[path] = tree.getpath(element)

            else:
                if element.attrib:
                    xpath_base = get_element_xpath(element)
                    credit_dict[xpath_base] = tree.getpath(element)
                    for e,val in element.attrib.items():
                        xpath = xpath_base + ' / ' + '@' + e
        #                 print(f'-->{xpath}')
                        credit_dict[xpath] = tree.getpath(element)
                else:
                    xpath = get_element_xpath(element)#defined on top
                    credit_dict[xpath] = tree.getpath(element)
    c_h = credit_dict
    return c_h
def credit_or_invoice(file_path):
#     print('********')
#     print(file_path)
    tree = etree.parse(file_path)
    target_root = tree.getroot()
    for index,e in enumerate(target_root.iter()):
        if index ==11:
            if 'Credit' in e.text:
                return 'credit'
            elif 'Invoice' in e.text:
                return 'invoice'
            else:
                return 'undefined'
def proper_mapping_final(file_path):

    if credit_or_invoice(file_path) == 'credit':
        r_h = get_mapping_for_xml_credit(file_path)
        return r_h
    elif credit_or_invoice(file_path) == 'invoice':
        r_h = get_mapping_for_xml_invoice(file_path)
        return r_h
    else:
        return None
def get_element_xpath(element):
    path = [element.tag.split('}')[-1]]
    for parent in element.iterancestors():
        parent_tag = parent.tag.split('}')[-1]  
        path.insert(0, parent_tag)
    return ' / '.join(path)
# def handle_attrib(atrib):#maybe store values just in case later ,i njust need the path in this case
#     for key in atrib:
#         key_val = key
#     key_val = '@' + key_val
#     return key_val
def read_xml_file(file_path_xml):
    result_xml_dict={}
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
#     pattern = r'\{.*?\}'
    for element in target_root.iter():
        if element.attrib:
            xpath_base = get_element_xpath(element)
            for e,val in element.attrib.items():
                xpath = xpath_base + ' / ' + '@' + e
#                 print(f'-->{xpath}')
                result_xml_dict[xpath] = tree.getpath(element)
        else:
            xpath = get_element_xpath(element)
            result_xml_dict[xpath] = tree.getpath(element)
    return result_xml_dict
        
def get_element_full_path(tree,element):
    return tree.getpath(element)

def read_xml_file_2(file_path_xml):
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
    for element in target_root.iter():
        xpath = tree.getpath(element)
        print(f"XPath: {xpath}")
def load_base_value_iden(file_path):
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            variable_loaded = "11e1d6e4-74ba-49cc-9b55-1"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        print('creating file...')
        variable_to_store = "11e1d6e4-74ba-49cc-9b55-1"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store
        
    return variable_loaded
def save_base_value_iden(file_path,variable_to_store):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
def increment_value_iden(value,file_path):

    str_last = value.split('-')[-1]
    last= int(str_last)
    last+=1
    
    exclude_last = value.split('-')[:-1]
    temp = '-'.join(exclude_last)

    new_identifier = temp + '-' + str(last)
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = new_identifier
#             print('writing')
#             print(variable_to_store)
            file.write(variable_to_store)
    else:
        print('load base value first')
def adjust_identifier(root_copy,filepath_base_identifier):
    base_identifier = load_base_value_iden(filepath_base_identifier)

    for index,e in enumerate(root_copy.iter()):
        if index == 10:#rsender identifier:
            e.text = base_identifier
            increment_value_iden(base_identifier,filepath_base_identifier)
def extract_out_second_half(actual_xpath):#to be attachrd to path_base
    temp = actual_xpath.split('/')[-1]
    return temp
def handle_multiple(current_test_case,test_case_id,r_h,target_root):
    print(f' {test_case_id}  ')
    
    hold_1 = prepare_data_for_multiple(current_test_case)
    print('------------')
    print(hold_1)
    print('   ')
    field_1 = hold_1[0][0]
    base_path_field_1 = handle_base_path(field_1)
    
    
    print('------------')
    print('   ')
    temp_base_path = base_path_field_1
    for index,each_case in enumerate(hold_1):
        path_base = temp_base_path + f'[{index+1}]' + '/'
        for each_path in each_case:
            base_path_each_path = handle_base_path(each_path)
            if base_path_each_path == base_path_field_1:
                
                print('   ')
                print(f'----<<--Ammend base path for {each_path} ->>-----')
                if each_path in r_h:
                    actual_xpath = r_h[each_path]
                    temp = extract_out_second_half(actual_xpath)
                    path_base_final = path_base + temp
                    print(f'----<<--actual amended path ==>{path_base_final}')
                    target_element = target_root.xpath(path_base_final,namespaces = namespace_map)
                    print(f'----<<--actual element==>{target_element}')
                    print('   ')
                    print(f'----<<--actual path {each_path}')
                    if '@' in each_path:
                        print('   ')
                        print('------change attrib------')
                        if len(target_element)>0:
                            print(f'----<<--actual element value==>{target_element[0].attrib}')
                            print('   ')
                        else:
                             print(f'--cannot get--<<--{each_path} | {target_element} is empty ')
                    else:
                        print('   ')
                        print('------change element------')
                        if len(target_element)>0:
                            print(f'----<<--actual element text==>{target_element[0].text}')
                        else:
                             print(f'---cannot get-<<--{each_path} | {target_element} is empty ')
                    print('   ')
            else:#base_path_each_path != base_path_field_1:
                print('   ')
                print(f'----<<--just change straight away {each_path} ->>')
                print('   ')
                if '@' in each_path:
                    print('   ')
                    print('------change attrib------')
                    print('   ')
                else:
                    print('   ')
                    print('------change element------')
                    print('   ')
                
                
    print('   ')
    print('------------')
    print('   ')
def handle_base_path(s):
    start = r_h[s]
#     print(f'start {start}')
    a= start
    specific_elem = a.split('/')[-1]
#     print(f'specific elem{specific_elem}')
    temp = a.split('/')[:-1]
#     print('temp',temp)
    temp_a = '/'.join(temp)
#     print('temp_a',temp_a)

    temp_b = temp[-1].split('[')[0]
    temp_c = temp
    temp_c[-1] = temp_b
    temp_d = '/'.join(temp_c)
#     print('temp_d',temp_d)
#     print('   ')
    return temp_d
            
def evaluate_if_got_multiple(current_test_case):
    occur_again = False
    for index,fields in enumerate(current_test_case):
        xpath_find = fields[0].strip()
        if index == 0:
            field_one = xpath_find
        if (xpath_find == field_one) and index != 0:
            occur_again = True
            return True
    return occur_again
def prepare_data_for_multiple(current_test_case):
    holding = []
    for index,fields in enumerate(current_test_case):
        
        path_to_store = fields[0]

        if index == 0:
            temp = []
            field_one = path_to_store
            temp.append(path_to_store)
            
        elif (path_to_store == field_one) and (index != 0):
            holding.append(temp)
            temp = []

        
            temp.append(path_to_store)
        else:
#             print('  ----  ')
#             print('    ')
#             print(temp)
#             print('  ----  ')
#             print('    ')
            temp.append(path_to_store)
    holding.append(temp)
    return holding
def fix_space_typo(temp):
    temp_2 = temp.split("/")
    for index,i in enumerate(temp_2):
        temp_2[index] = i.strip()
    temp_3 = ' / '.join(temp_2)
    return temp_3
def adjust_cac(input_string):
    a='tax[2]'
    b = str(int(a[-2])-1)
    prefix = a[:-2]
    return prefix + b + a[-1]

def adjust_customisation_id(root_copy,value):
     for index,e in enumerate(root_copy.iter()):
        if index == 31:
            print("----")
            print("--customisation--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")

            
        
def adjust_profile_id(root_copy,value):
    for index,e in enumerate(root_copy.iter()):
        if index == 33:
            print("----")
            print("--profile--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")


def insert_for_attribute(target_element,insert_value,each_path):
    print('   ')
    print('------change attrib------')
    if len(target_element)>0:

        print(f'----<<--actual element value==>{target_element[0].attrib} type = {type(target_element[0].attrib)}')
        for key, value in target_element[0].attrib.items():
            print(f'val = {value}')
            target_element[0].attrib[key] = insert_value
        print(f'----<<--new attribute value==>{target_element[0].attrib}')

    else:
         print(f'--cannot get--<<--{each_path} | {target_element} is empty ')
    return target_element
def adjust_cac(input_string):
    a='tax[2]'
    b = str(int(a[-2])-1)
    prefix = a[:-2]
    return prefix + b + a[-1]

def insert_for_element(target_element,insert_value,each_path,actual_xpath):
    
    print('   ')
    print('------change element------')
    if len(target_element)>0:
        print(f'----<<--actual element text==>{target_element[0].text}')
        target_element[0].text = insert_value
        
        print(f'----<<--new element text==>{target_element[0].text}')
#         tree = etree.ElementTree(original_tree_copy)
# #         indexed_filename =sheet_name + 'Test case ID_' + str(test_case_id) + '_' + rule_id + '.xml'
#         indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
#         filepath = os.path.join(folder_name, indexed_filename)
    else:
        print(f'---cannot get-<<--{each_path} | {target_element} is empty ')
    return target_element

def handle_descriptions(desc,index):#like a custom immigration gate,right before writng to file,remove if element
    if 'Field does not exist' in desc and index==0:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('                                   ')
        
        print('RUMBLE')
        
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('                                   ')
    else:
        pass
#still there
#takes in desc and fieldone path.resolves into xpath
#remove element if still there at the end for filed does not exist
#change from 'nan' to '' if field exist with missingn value,adjust value_store first element as thats waht
#field description is referring to

def  delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path):
    attrib_yes = False
  
    if '@' in each_path:
        attrib_yes = True
        
    if 'Field does not exist' in desc:
        print(f'%%%%%%%%xpath find{actual_xpath_absolute} %%%%%%%%')
        print('                 ')
        print('Targeting element......')
        element_temp = target_root_copy.xpath(actual_xpath_absolute, namespaces=namespace_map)
        if len(element_temp)>0:
            element = element_temp[0]
            if attrib_yes:
                print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
                del element.attrib[key_attrib]
                print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
#                 key_attrib = xpath_find.split('@')[-1]
#                 print(f'removing attrib {xpath_find}')
#                 print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
#                 del element.attrib[key_attrib]
#                 print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
            else:
                print('%%%%%%%% Remove element %%%%%%%%')
                parent = element.getparent()

                if element.getparent() is not None:
                    element.getparent().remove(element)

                print('------------')
        else:
            print('xpath ')
        return target_root_copy
    else:
        print('%%%%%%%no attribute to remove -function : delete_element_attrib exiting%%%%%%%%%')
        print('                  ')
        return target_root_copy


def extract_out_second_half(actual_xpath):#to be attachrd to path_base
    temp = actual_xpath.split('/')[-1]
    return temp
def handle_base_path(s,r_h):
    start = r_h[s]
#     print(f'start {start}')
    a= start
    specific_elem = a.split('/')[-1]
#     print(f'specific elem{specific_elem}')
    temp = a.split('/')[:-1]
#     print('temp',temp)
    temp_a = '/'.join(temp)
#     print('temp_a',temp_a)

    temp_b = temp[-1].split('[')[0]
    temp_c = temp
    temp_c[-1] = temp_b
    temp_d = '/'.join(temp_c)
#     print('temp_d',temp_d)
#     print('   ')
    return temp_d
            
def evaluate_if_got_multiple(current_test_case):
    occur_again = False
    for index,fields in enumerate(current_test_case):
        xpath_find = fields[0].strip()
        if index == 0:
            field_one = xpath_find
        if (xpath_find == field_one) and index != 0:
            occur_again = True
            return True
    return occur_again
def prepare_data_for_multiple(current_test_case):
    holding = []
    values_store=[]
    for index,fields in enumerate(current_test_case):
        
        path_to_store = fields[0]
        value = fields[1]
        values_store.append(value)
        if index == 0:
            temp = []
            
            field_one = path_to_store
            temp.append(path_to_store)
            
        elif (path_to_store == field_one) and (index != 0):
            holding.append(temp)
            temp = []

        
            temp.append(path_to_store)
        else:
#             print('  ----  ')
#             print('    ')
#             print(temp)
#             print('  ----  ')
#             print('    ')
            temp.append(path_to_store)
    holding.append(temp)
    return holding , values_store
def add_index_before_slash(input_string,index):
    # Find the position of the first '/'
    
    
    slash_index = input_string.find('/')
    
    # Check if '/' is found
    if slash_index == -1:
        # Return the original string if no '/' is found
        return input_string
    
    # Insert '[1]' before the first '/'
    if re.search(r'\[\d+\]', input_string):
        # Replace the existing '[]' with the new index
        modified_string = re.sub(r'\[\d+\]', f'[{index}]', input_string)
    else:
        # Insert '[index]' before the first '/'
        modified_string = input_string[:slash_index] + f'[{index}]' + input_string[slash_index:]
    
    return modified_string

def cac_count(s,substr):
    return s.count(substr)


def fix_get_second_cac(s,index,helper):
    
    if helper != 'Special' or cac_count(s,'cac') <= 1:
        print('no changees.......')
        return s
    else:
        t_split = s.split('cac')
        to_fix=t_split[-2]
        t_split[-1] = add_index_before_slash(t_split[-1],1)
        t_split[-2] = add_index_before_slash(to_fix,index)

        return 'cac'.join(t_split)

        
        


def handle_multiple(current_test_case,test_case_id,r_h,target_root,desc,helper):
    print(f' {test_case_id}  ')
    base_path_field_1_a = []
    whole_count_pos = 0
    hold_1 , value_store = prepare_data_for_multiple(current_test_case)
    print('------------')
    print(hold_1)
    print('   ')
    field_1 = hold_1[0][0]
    base_path_field_1 = handle_base_path(field_1,r_h)
    base_path_field_1_a.append(handle_base_path(field_1,r_h))
    print('88888')
    print(base_path_field_1)
    print('   ')
    value_store_key = 0
    temp_base_path = base_path_field_1
    target_root_copy = copy.deepcopy(target_root)
    

    
    for index,each_case in enumerate(hold_1):
        
        

        path_base = temp_base_path + f'[{index+1}]' + '/'

        for each_path in each_case:
            
            print('+++++++++++++++++++++++++')
            

            
            insert_value = str(value_store[value_store_key])
            print('+++++++++++++++++++++++++')
            print('---------------')
            print('   ')
            
            print(f'insert value = {value_store[value_store_key]}')
            if 'Field exists with missing value' in desc and index == 0:
                
                print(f'old insert value {insert_value} type is {type(insert_value)}')
                print(f'##Description = {desc}##')
                insert_value = ''
                print(f'new insert value {insert_value}')
            value_store_key +=1
      

            base_path_each_path = handle_base_path(each_path,r_h)
            print('   ')
            print('========')
            print(base_path_each_path)
            print('========')
            print('   ')
            if base_path_each_path == base_path_field_1:

                print('   ')
                if each_path in r_h:
                    actual_xpath = r_h[each_path]
                    temp = extract_out_second_half(actual_xpath)
                    
                    actual_xpath_absolute = path_base + temp
                    print('base_each_Path = base path field 1')
                    print('=======ACTUAL XPATH========')
#                     actual_xpath_absolute = fix_get_second_cac(actual_xpath_absolute,index+1,helper)
                    print(f'----<<--actual amended base path ==>{actual_xpath_absolute}')
                    
                    target_element = target_root_copy.xpath(actual_xpath_absolute,namespaces = namespace_map)

                    if '@' in each_path:
                        print(f'target element ==>{target_element}')
                        print(f'>>>>>>target_element.attrib = {target_element[0].attrib}')
                        print(f'>>>>>>target_element.text = {target_element[0].text}')
                        target_element = insert_for_attribute(target_element,insert_value,each_path)
                        print('             ')
                        
                        print('   ')
                    else:
                        print('  inserting value for elment.... \n')
                        target_element = insert_for_element(target_element,insert_value,each_path,actual_xpath_absolute)
                    if whole_count_pos == 0:
                        print(']]]]]]]]]]]]]]]]]base_path_each_path == base_path_field_1]]]]]]]]]]]]]]]]]]')
                        print(f'each path = {each_path}')
                        print(f'{actual_xpath_absolute}')
                        target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path)
                        print('<<<<last>>>')
      

            else:#base_path_each_path != base_path_field_1:
                    print('   ')
                    print(f'----<<--just change straight away {each_path} index == {whole_count_pos}->>')
                    print('   ')
                    if each_path in r_h:
                        actual_xpath = r_h[each_path]
#                     actual_xpath = fix_ambig_1(helper,actual_xpath_absolute,index)
                    print('=======ACTUAL XPATH========')
#                     actual_xpath = fix_get_second_cac(actual_xpath,index+1,helper)
                    print(f'acutal path : {actual_xpath}')
                    
                    target_element = target_root_copy.xpath(actual_xpath,namespaces = namespace_map)
                    print('    ')
                    print('===============')
                    print(f'acutal path : {actual_xpath}')
                    
                    
                    print('   ')
                    if '@' in each_path:
                        print('   ')
                        print(f'target element {target_element}')
                        print('------change attrib------')
                        target_element = insert_for_attribute(target_element,insert_value,each_path)
                       
                        print('   ')
                    else:
                        print('   ')
                        print(f'target element {target_element}')
                        print('------change element------')
                        target_element = insert_for_element(target_element,insert_value,each_path,actual_xpath)
                        

                        print('   ')
                    if whole_count_pos == 0:
                        print(']]]]]]]]]]]]]]]base_path_each_path != base_path_field_1]]]]]]]]]]]]]]]]]]]]')
                        print(f'each path = {each_path}')
                        print(f'{actual_xpath_absolute}')
                        target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath,r_h)
                        print('<<<<last>>>')
#             if whole_count_pos == 0:

#                 print(f'{each_path}')
#                 print(f'{ea}')
#                 if '@' in each_path:
#                     pass
#                 print('<<<<last>>>')
   
#
##
#
            whole_count_pos += 1

#     print(value_store_key)
#     print('             ')
#     print(value_store)
    return target_root_copy


In [21]:
output_folder = '/Users/cheongray/iras_api_clean_1/5_Record_Validation'
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

output_file = os.path.join(output_folder, 'df_full.csv')
output_file_original = os.path.join(output_folder, 'df_full_back_up.csv')
if os.path.exists(output_file):
    os.remove(output_file)
if os.path.exists(output_file_original):
    os.remove(output_file_original)


In [22]:
file_path = r"/Users/cheongray/iras_api_pytest_1/AP_SP_files/AP_SP_FINAL_5.xlsx"#download to here from emails

path_to_sample_xml_files_folder = r"/Users/cheongray/iras_api_clean_1/sample_xml_files"

sheet_name = '5_Record Validation'
df_s = pd.read_excel(file_path, sheet_name=sheet_name , keep_default_na=False, na_values=['NA'])
df_s_temp = df_s.copy()
df_s_temp = df_s_temp.fillna('NA')

df_s_temp['Test Case ID'] = df_s_temp.index.astype(float)#REMINDER,now test_case_id is 1_1
# df_s_peppol = df_s_temp[(df_s_temp['Standard'] == 'PEPPOL') & (df_s_temp['Document Type'] == 'Invoice')].copy()
# df_s_peppol['Test Case ID'] = df_s_peppol['Test Case ID'] + 1
# df_s_peppol.reset_index(drop=True)
df_s_temp['Test Case ID'] = df_s_temp['Test Case ID'] + 1
df_s_temp.to_csv(output_file, index=False)
df_s_temp.to_csv(output_file_original, index=False)

In [23]:

df_s_temp_subset = df_s_temp.copy()

unique_descriptions = set()

target_elements_syn = []

for index,row in df_s_temp_subset.iterrows():
    filled_columns = row.dropna().index
    test_case_id = f' {index}'
    descript = row['Test Case Description']
#     print(f'Test case {index+1}')
#     print(f'Test case description : {descript}')
    unique_descriptions.add(descript)#just to build function_mapping
    temp={}
    temp['Rule ID'] = row['Rule ID']
    temp['Standard'] = row['Standard']
    temp['Document Type'] = row['Document Type']
    temp['BIS/PINT'] = row['BIS/PINT']
    temp['Test Case ID'] = int(row['Test Case ID'])
    id = int(row['Test Case ID'])
    temp['descript'] = descript
    temp['Expected Result'] = row['Expected Result Response (for 400 response)']
    temp['Status'] = row['Expected Result']
    temp['Fields'] = []
    temp['Sample'] = row['Sample']
    for i, col in enumerate(filled_columns, start=1):
        field_name = f'Field {i}'
        if field_name in df_s_temp_subset.columns:
            # Process the data as needed
#             print(f'Column: {field_name}, Value: {row[field_name]}')
   
            
            temp_2 = [] #each field will be one array,can do dict but mor steps when want access
            if not pd.isna(row[field_name]):
                field_value = field_name + ' Value'
                field_name_insert = fix_space_typo(row[field_name])
                temp_2.append(field_name_insert)
                temp_2.append(row[field_value])
                if field_name_insert == '':
                    continue
                else:
                    temp['Fields'].append(temp_2)
        
    target_elements_syn.append(temp)

            


In [24]:
# path_to_sample_xml_files = '/Users/cheongray/iras_api_pytest_1/sample_xml_files'
#gets the needed files for 2_Conditional_Input_Validation
file_paths_xml_files = {}

for i,test_case in enumerate(target_elements_syn):
    
    sample_to_use = test_case['Sample']
    
    num = sample_to_use.split(' ')[-1]
    file_name = 'sample' + '_' + num + '.xml'
    
    path_to_sample_xml_files = path_to_sample_xml_files_folder + '/' + file_name
    
    file_paths_xml_files[sample_to_use] = path_to_sample_xml_files


In [25]:
def adjust_customisation_id(root_copy,value):
     for index,e in enumerate(root_copy.iter()):
        if index == 31:
            print("----")
            print("--customisation--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")

            
        
def adjust_profile_id(root_copy,value):
    for index,e in enumerate(root_copy.iter()):
        if index == 33:
            print("----")
            print("--profile--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")


def insert_for_attribute(target_element,insert_value,each_path):
    print('   ')
    print('------change attrib------')
    if len(target_element)>0:

        print(f'----<<--actual element value==>{target_element[0].attrib} type = {type(target_element[0].attrib)}')
        for key, value in target_element[0].attrib.items():
            print(f'val = {value}')
            target_element[0].attrib[key] = insert_value
        print(f'----<<--new attribute value==>{target_element[0].attrib}')

    else:
         print(f'--cannot get--<<--{each_path} | {target_element} is empty ')
    return target_element
def adjust_cac(input_string):
    a='tax[2]'
    b = str(int(a[-2])-1)
    prefix = a[:-2]
    return prefix + b + a[-1]

def insert_for_element(target_element,insert_value,each_path,actual_xpath):
    
    print('   ')
    print('------change element------')
    if len(target_element)>0:
        print(f'----<<--actual element text==>{target_element[0].text}')
        target_element[0].text = insert_value
        
        print(f'----<<--new element text==>{target_element[0].text}')
#         tree = etree.ElementTree(original_tree_copy)
# #         indexed_filename =sheet_name + 'Test case ID_' + str(test_case_id) + '_' + rule_id + '.xml'
#         indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
#         filepath = os.path.join(folder_name, indexed_filename)
    else:
        print(f'---cannot get-<<--{each_path} | {target_element} is empty ')
    return target_element

def handle_descriptions(desc,index):#like a custom immigration gate,right before writng to file,remove if element
    if 'Field does not exist' in desc and index==0:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('                                   ')
        
        print('RUMBLE')
        
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('                                   ')
    else:
        pass
#still there
#takes in desc and fieldone path.resolves into xpath
#remove element if still there at the end for filed does not exist
#change from 'nan' to '' if field exist with missingn value,adjust value_store first element as thats waht
#field description is referring to

def  delete_element_attrib(desc,target_root,actual_xpath_absolute,r_h,each_path):
    attrib_yes = False
    target_root_copy = copy.deepcopy(target_root)
    if '@' in each_path:
        attrib_yes = True
        
    if 'Field does not exist' in desc:
        print(f'%%%%%%%%xpath find{actual_xpath_absolute} %%%%%%%%')
        print('                 ')
        print('Targeting element......')
        element_temp = target_root_copy.xpath(actual_xpath_absolute, namespaces=namespace_map)
        if len(element_temp)>0:
            element = element_temp[0]
            if attrib_yes:
                print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
                del element.attrib[key_attrib]
                print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
#                 key_attrib = xpath_find.split('@')[-1]
#                 print(f'removing attrib {xpath_find}')
#                 print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
#                 del element.attrib[key_attrib]
#                 print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
            else:
                print('%%%%%%%% Remove element %%%%%%%%')
                parent = element.getparent()

                if element.getparent() is not None:
                    element.getparent().remove(element)

                print('------------')
        else:
            print('xpath ')
        return target_root_copy
    else:
        print('%%%%%%%no attribute to remove -function : delete_element_attrib exiting%%%%%%%%%')
        print('                  ')
        return target_root_copy


def extract_out_second_half(actual_xpath):#to be attachrd to path_base
    temp = actual_xpath.split('/')[-1]
    return temp
def handle_base_path(s,r_h):
    start = r_h[s]
#     print(f'start {start}')
    a= start
    specific_elem = a.split('/')[-1]
#     print(f'specific elem{specific_elem}')
    temp = a.split('/')[:-1]
#     print('temp',temp)
    temp_a = '/'.join(temp)
#     print('temp_a',temp_a)

    temp_b = temp[-1].split('[')[0]
    temp_c = temp
    temp_c[-1] = temp_b
    temp_d = '/'.join(temp_c)
#     print('temp_d',temp_d)
#     print('   ')
    return temp_d
            
def evaluate_if_got_multiple(current_test_case):
    occur_again = False
    for index,fields in enumerate(current_test_case):
        xpath_find = fields[0].strip()
        if index == 0:
            field_one = xpath_find
        if (xpath_find == field_one) and index != 0:
            occur_again = True
            return True
    return occur_again
def prepare_data_for_multiple(current_test_case):
    holding = []
    values_store=[]
    for index,fields in enumerate(current_test_case):
        
        path_to_store = fields[0]
        value = fields[1]
        values_store.append(value)
        if index == 0:
            temp = []
            
            field_one = path_to_store
            temp.append(path_to_store)
            
        elif (path_to_store == field_one) and (index != 0):
            holding.append(temp)
            temp = []

        
            temp.append(path_to_store)
        else:
#             print('  ----  ')
#             print('    ')
#             print(temp)
#             print('  ----  ')
#             print('    ')
            temp.append(path_to_store)
    holding.append(temp)
    return holding , values_store
def add_index_before_slash(input_string,index):
    # Find the position of the first '/'
    
    
    slash_index = input_string.find('/')
    
    # Check if '/' is found
    if slash_index == -1:
        # Return the original string if no '/' is found
        return input_string
    
    # Insert '[1]' before the first '/'
    if re.search(r'\[\d+\]', input_string):
        # Replace the existing '[]' with the new index
        modified_string = re.sub(r'\[\d+\]', f'[{index}]', input_string)
    else:
        # Insert '[index]' before the first '/'
        modified_string = input_string[:slash_index] + f'[{index}]' + input_string[slash_index:]
    
    return modified_string

def cac_count(s,substr):
    return s.count(substr)


def fix_get_second_cac(s,index,helper):
    
    if helper != 'Special' or cac_count(s,'cac') <= 1:
        print('no changees.......')
        return s
    else:
        t_split = s.split('cac')
        to_fix=t_split[-2]
        t_split[-1] = add_index_before_slash(t_split[-1],1)
        t_split[-2] = add_index_before_slash(to_fix,index)

        return 'cac'.join(t_split)

        
        


def handle_multiple(current_test_case,test_case_id,r_h,target_root,desc,helper):
    print(f' {test_case_id}  ')
    base_path_field_1_a = []
    whole_count_pos = 0
    hold_1 , value_store = prepare_data_for_multiple(current_test_case)
    print('------------')
    print(hold_1)
    print('   ')
    field_1 = hold_1[0][0]
    base_path_field_1 = handle_base_path(field_1,r_h)
    base_path_field_1_a.append(handle_base_path(field_1,r_h))
    print('88888')
    print(base_path_field_1)
    print('   ')
    value_store_key = 0
    temp_base_path = base_path_field_1
    target_root_copy = copy.deepcopy(target_root)
    

    
    for index,each_case in enumerate(hold_1):
        
        

        path_base = temp_base_path + f'[{index+1}]' + '/'

        for each_path in each_case:
            
            print('+++++++++++++++++++++++++')
            

            
            insert_value = str(value_store[value_store_key])
            print('+++++++++++++++++++++++++')
            print('---------------')
            print('   ')
            
            print(f'insert value = {value_store[value_store_key]}')
            if 'exists with missing value' in desc and index == 0:
                
                print(f'old insert value {insert_value} type is {type(insert_value)}')
                print(f'##Description = {desc}##')
                insert_value = ''
                print(f'new insert value {insert_value}')
            value_store_key +=1
      

            base_path_each_path = handle_base_path(each_path,r_h)
            print('   ')
            print('========')
            print(base_path_each_path)
            print('========')
            print('   ')
            if base_path_each_path == base_path_field_1:

                print('   ')
                if each_path in r_h:
                    actual_xpath = r_h[each_path]
                    temp = extract_out_second_half(actual_xpath)
                    
                    actual_xpath_absolute = path_base + temp
                    print('base_each_Path = base path field 1')
                    print('=======ACTUAL XPATH========')
#                     actual_xpath_absolute = fix_get_second_cac(actual_xpath_absolute,index+1,helper)
                    print(f'----<<--actual amended base path ==>{actual_xpath_absolute}')
                    
                    target_element = target_root_copy.xpath(actual_xpath_absolute,namespaces = namespace_map)

                    if '@' in each_path:
                        print(f'target element ==>{target_element}')
                        print(f'>>>>>>target_element.attrib = {target_element[0].attrib}')
                        print(f'>>>>>>target_element.text = {target_element[0].text}')
                        target_element = insert_for_attribute(target_element,insert_value,each_path)
                        print('             ')
                        
                        print('   ')
                    else:
                        print('  inserting value for elment.... \n')
                        target_element = insert_for_element(target_element,insert_value,each_path,actual_xpath_absolute)
                    if whole_count_pos == 0:
                        print(']]]]]]]]]]]]]]]]]base_path_each_path == base_path_field_1]]]]]]]]]]]]]]]]]]')
                        print(f'each path = {each_path}')
                        print(f'{actual_xpath_absolute}')
                        target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path)
                        print('<<<<last>>>')
      

            else:#base_path_each_path != base_path_field_1:
                    print('   ')
                    print(f'----<<--just change straight away {each_path} index == {whole_count_pos}->>')
                    print('   ')
                    if each_path in r_h:
                        actual_xpath = r_h[each_path]
#                     actual_xpath = fix_ambig_1(helper,actual_xpath_absolute,index)
                    print('=======ACTUAL XPATH========')
#                     actual_xpath = fix_get_second_cac(actual_xpath,index+1,helper)
                    print(f'acutal path : {actual_xpath}')
                    
                    target_element = target_root_copy.xpath(actual_xpath,namespaces = namespace_map)
                    print('    ')
                    print('===============')
                    print(f'acutal path : {actual_xpath}')
                    
                    
                    print('   ')
                    if '@' in each_path:
                        print('   ')
                        print(f'target element {target_element}')
                        print('------change attrib------')
                        target_element = insert_for_attribute(target_element,insert_value,each_path)
                       
                        print('   ')
                    else:
                        print('   ')
                        print(f'target element {target_element}')
                        print('------change element------')
                        target_element = insert_for_element(target_element,insert_value,each_path,actual_xpath)
                        

                        print('   ')
                    if whole_count_pos == 0:
                        print(']]]]]]]]]]]]]]]base_path_each_path != base_path_field_1]]]]]]]]]]]]]]]]]]]]')
                        print(f'each path = {each_path}')
                        print(f'{actual_xpath_absolute}')
                        target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath,r_h)
                        print('<<<<last>>>')
#             if whole_count_pos == 0:

#                 print(f'{each_path}')
#                 print(f'{ea}')
#                 if '@' in each_path:
#                     pass
#                 print('<<<<last>>>')
   
#
##
#
            whole_count_pos += 1

#     print(value_store_key)
#     print('             ')
#     print(value_store)
    return target_root_copy


In [26]:
folder_name = output_folder + '/xml_files_for'+'_'+sheet_name

filepath_base_identifier = '/Users/cheongray/iras_api_make_xml/identifier_store/stored_base_identifier_conditional_validation.txt'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)
    os.makedirs(folder_name)
    
namespace_map = {
    'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    }
  
def adjust_customisation_id(root_copy,value):
     for index,e in enumerate(root_copy.iter()):
        if index == 31:
            print("----")
            print("--customisation--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")
  
def adjust_profile_id(root_copy,value):
    for index,e in enumerate(root_copy.iter()):
        if index == 33:
            print("----")
            print("--profile--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")

c=0
error_prone_files = []
d=0
error_in_file = []
error_in_file_store = []
files_made = []
attrib=0
for i,test_case in enumerate(target_elements_syn):
    
    current_test_case = test_case['Fields']
    test_case_id = test_case['Test Case ID']
    desc = test_case['descript']
    sample_to_use = test_case['Sample']
    standard  = test_case['Standard']
    biz_pint = test_case['BIS/PINT']
    rule_id = test_case['Rule ID']
#     helper = test_case['Helper']
    f_p = file_paths_xml_files[sample_to_use]
    print(f'###################{test_case_id}##########################')
    print(' ')
    print(f'{test_case_id}  using {f_p}')
    print('   ')
    r_h = proper_mapping_final(f_p)
#     print(f'==helper_value===={helper}===={type(helper)}=')
    if not r_h:
        print(f'cannot make r_h : {test_case_id}  sample :{sample_to_use}')
        continue
    tree = etree.parse(f_p)
    target_root = tree.getroot()
#     print(f'test case id :{test_case_id}')
#     print(desc)
    original_tree_copy = copy.deepcopy(target_root)
    if standard == 'PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1'
            adjust_customisation_id(original_tree_copy,value_cus)

            
    if standard == 'Non-PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)
    adjust_identifier(original_tree_copy,filepath_base_identifier)
    lenght_of_fields = len(current_test_case)
    occur_again = evaluate_if_got_multiple(current_test_case)
    if occur_again:
        
        print('ocur again ')
        c+=1
        temp = []
        temp.append(test_case['Test Case ID'])
        temp.append(test_case['descript'])
        print(' ')
        error_prone_files.append(temp)

        target_root = handle_multiple(current_test_case,test_case_id,r_h,original_tree_copy,desc,'temp')
        tree = etree.ElementTree(target_root)
        indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
        filepath = os.path.join(folder_name, indexed_filename)
        print(f'writing {indexed_filename} to folder......')
        tree.write(filepath, encoding='utf-8', xml_declaration=True)
        
    else:
        print('not occur again')
        d+=1
        temp = {}
        temp[test_case_id] = []
        for index,fields in enumerate(current_test_case):
            attrib_yes = False
            xpath_find = fields[0].strip()

            field_value = fields[1]
            if pd.isna(field_value):
                print('    ')
                print('++is NAN !!!!!!!!++')
                print('     ')

            if '@' in xpath_find:
                    attrib_yes = True

            if xpath_find in r_h:
                xpath_target = r_h[xpath_find]
                print(f'====={xpath_find}=======')
                if original_tree_copy.xpath(xpath_target, namespaces=namespace_map):
                    element = original_tree_copy.xpath(xpath_target, namespaces=namespace_map)
                    if not attrib_yes:
                        print('====text====')
                        print(f'original_text {element[0].text}')
                        element[0].text = str(field_value)
                        print(f'new text = {element[0].text}')
                        print('  ')

                    else:
                        print('===attribute====')
                        insert_for_attribute(element,str(field_value),xpath_find)
                        print(' ')
            else:#+++
                
                temp[test_case_id].append(xpath_find)
                print(f'--->test case:{test_case_id} | uses {sample_to_use} key not found for {xpath_find}')
        if len(temp[test_case_id])>=1:
            error_in_file.append(temp[test_case_id])
                
        tree = etree.ElementTree(original_tree_copy)
    #         indexed_filename =sheet_name + 'Test case ID_' + str(test_case_id) + '_' + rule_id + '.xml'
        indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
        filepath = os.path.join(folder_name, indexed_filename)
        print(f'writing {indexed_filename }')
        tree.write(filepath, encoding='utf-8', xml_declaration=True)
        files_made.append(test_case_id)
        print(f'###################{test_case_id}##########################')
        print('       ')
        print('       ')
print('c=',c)



###################1##########################
 
1  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
   
----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x13ac262c0> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x13ad159c0> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
ocur again 
 
 1  
------------
[['ubl:Invoice / cac:TaxTotal / cbc:TaxAmount', 'ubl:Invoice / cac:TaxTotal / cbc:TaxAmount / @currencyID'], ['ubl:Invoice / cac:TaxTotal / cbc:TaxAmount', 'ubl:Invoice / cac:TaxTotal / cbc:TaxAmount / @currencyID']]
   
88888
/*/*[2]/cac:TaxTotal
   
+++++++++++++++++++++++++
+++++++++++++++++++++++++
---------------
   
insert value = 45
   
/*/*[2]/cac:TaxTotal
   
   
base_each_Pa

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x13b23d300> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x12b3b0740> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
ocur again 
 
 29  
------------
[['ubl:Invoice / cac:TaxTotal / cbc:TaxAmount', 'ubl:Invoice / cac:TaxTotal / cbc:TaxAmount / @currencyID'], ['ubl:Invoice / cac:TaxTotal / cbc:TaxAmount', 'ubl:Invoice / cac:TaxTotal / cbc:TaxAmount / @currencyID']]
   
88888
/*/*[2]/cac:TaxTotal
   
+++++++++++++++++++++++++
+++++++++++++++++++++++++
---------------
   
insert value = -45
   
/*/*[2]/cac:TaxTotal
   
   
base_each_Path = base path field 1
=======ACTUAL XPATH========
----<<--actual amended base path ==>/*/*[2]/cac:TaxTotal[1]/cbc:TaxAmount

ocur again 
 
 67  
------------
[['ubl:CreditNote / cac:TaxTotal / cbc:TaxAmount', 'ubl:CreditNote / cac:TaxTotal / cbc:TaxAmount / @currencyID'], ['ubl:CreditNote / cac:TaxTotal / cbc:TaxAmount', 'ubl:CreditNote / cac:TaxTotal / cbc:TaxAmount / @currencyID']]
   
88888
/*/*[2]/cac:TaxTotal
   
+++++++++++++++++++++++++
+++++++++++++++++++++++++
---------------
   
insert value = 0
   
/*/*[2]/cac:TaxTotal
   
   
base_each_Path = base path field 1
=======ACTUAL XPATH========
----<<--actual amended base path ==>/*/*[2]/cac:TaxTotal[1]/cbc:TaxAmount
  inserting value for elment.... 

   
------change element------
----<<--actual element text==>2374.05
----<<--new element text==>0
]]]]]]]]]]]]]]]]]base_path_each_path == base_path_field_1]]]]]]]]]]]]]]]]]]
each path = ubl:CreditNote / cac:TaxTotal / cbc:TaxAmount
/*/*[2]/cac:TaxTotal[1]/cbc:TaxAmount
%%%%%%%no attribute to remove -function : delete_element_attrib exiting%%%%%%%%%
                  
<<<<last>>>
+++++++++++++++++++++++++
+

ocur again 
 
 100  
------------
[['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID', 'ubl:Invoice / cac:InvoiceLine / cbc:ID'], ['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID', 'ubl:Invoice / cac:InvoiceLine / cbc:ID', 'ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID', 'ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID', 'ubl:Invoice / cbc:ProfileID']]
   
88888
/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory
   
+++++++++++++++++++++++++
+++++++++++++++++++++++++
---------------
   
insert value = SRLVG
   
/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory
   
   
base_each_Path = base path field 1
=======ACTUAL XPATH========
----<<--actual amended base path ==>/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory[1]/cbc:ID
  inserting value for elment.... 

   
------change element------
----<<--actual element text==>ZR
----<<--new element t

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x13ad50bc0> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x13b23d300> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
not occur again
=====ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
====text====
original_text SR
new text = SRCA-C
  
=====ubl:Invoice / cbc:ProfileID=======
====text====
original_text urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
new text = urn:fdc:peppol.eu:2017:poacc:Payables:01:1.0
  
=====ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
====text====
original_text SR
new text = SRCA-C
  
writing IRASC5-049_Test_case_139_5_Record Validation.xml
###################139####

...

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x13ad4aac0> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x13ad48380> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
not occur again
=====ubl:Invoice / cac:LegalMonetaryTotal / cbc:PayableRoundingAmount / @currencyID=======
===attribute====
   
------change attrib------
----<<--actual element value==>{'currencyID': 'SGD'} type = <class 'lxml.etree._Attrib'>
val = SGD
----<<--new attribute value==>{'currencyID': 'USD'}
 
writing IRASC5-075_Test_case_314_5_Record Validation.xml
###################314##########################
       
       
###################315##########################
 
315  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sam

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x13ad4aac0> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x13ad4aac0> element text ->urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0
  
----
----
not occur again
=====ubl:CreditNote / cac:TaxTotal / cbc:TaxAmount / @currencyID=======
===attribute====
   
------change attrib------
----<<--actual element value==>{'currencyID': 'SGD'} type = <class 'lxml.etree._Attrib'>
val = SGD
----<<--new attribute value==>{'currencyID': 'SGD'}
 
writing IRASC5-075_Test_case_357_5_Record Validation.xml
###################357##########################
       
       
###################358##########################
 
358  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml
   
----
--profile--
  
<Element {urn:oasis:name

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x13ad4aac0> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x13ad4a3c0> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
not occur again
=====ubl:Invoice / cac:AllowanceCharge / cbc:Amount / @currencyID=======
===attribute====
   
------change attrib------
----<<--actual element value==>{'currencyID': 'SGD'} type = <class 'lxml.etree._Attrib'>
val = SGD
----<<--new attribute value==>{'currencyID': 'MYR'}
 
writing IRASC5-075_Test_case_398_5_Record Validation.xml
###################398##########################
       
       
###################399##########################
 
399  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml
   
---

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x13ad4aac0> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x13ad48940> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
not occur again
=====ubl:Invoice / cac:InvoiceLine / cac:AllowanceCharge / cbc:Amount / @currencyID=======
===attribute====
   
------change attrib------
----<<--actual element value==>{'currencyID': 'SGD'} type = <class 'lxml.etree._Attrib'>
val = SGD
----<<--new attribute value==>{'currencyID': 'SGD'}
 
writing IRASC5-075_Test_case_443_5_Record Validation.xml
###################443##########################
       
       
###################444##########################
 
444  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sam

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x13a5c0e00> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x13a5c0800> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
not occur again
=====ubl:Invoice / cac:InvoiceLine / cac:Price / cac:AllowanceCharge / cbc:Amount / @currencyID=======
===attribute====
   
------change attrib------
----<<--actual element value==>{'currencyID': 'SGD'} type = <class 'lxml.etree._Attrib'>
val = SGD
----<<--new attribute value==>{'currencyID': 'SGD'}
 
writing IRASC5-075_Test_case_489_5_Record Validation.xml
###################489##########################
       
       
###################490##########################
 
490  using /Users/cheongray/iras_api_clean_1/sample_x

In [27]:
problems=[]
for i in error_prone_files:
    print(i[0])
    problems.append(i[0])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
75
76
77
78
79
80
83
84
85
86
87
88
91
92
93
94
95
96
99
100
101
102
103
104
107
108
109
110
111
112
115
116
117
118
119
120
123
124
125
126
127
128
131
132
133
134
135
136
